In [ ]:
import psi4
import fortecubeview as vis
import glob

## Vibrational frequency analysis can be used to verify a transition state structure.
## Here, we have a rough guess of a transition state structure of CNH -> HCN reaction.

In [ ]:
psi4.set_memory('500 MB')

CNH  = """
      0 1 
      C  0.0  0.0  0.0 
      N  0.0  1.16  0.0
      H  0.0  0.5  1.0
      """
vis.geom(xyz=CNH)

## We will optimize it first and obtain vibrational modes.

In [ ]:
TS = psi4.geometry(CNH)
psi4.core.set_output_file('TSoptimization.out')
psi4.set_options({'basis': '6-31g',
                  'reference': 'rhf',
                  'opt_type':'TS'})
energy, wfn = psi4.optimize('scf', molecule = TS,return_wfn = True)
print('Energy of the optimized water is %.7f' %energy)
vis.geom(molecule = TS)

In [ ]:
psi4.set_options({"normal_modes_write ":True,
                  "writer_file_label":"CNH"})
psi4.core.set_output_file('vib.out')
E, wfn = psi4.frequency('scf', molecule=TS, return_wfn=True)

freqs = [x for x in wfn.frequencies().np[:]]
print("The three normal mode vibrational frequencies are %.2f cm-1, %.2f cm-1, and %.2f cm-1" %(freqs[0], freqs[1], freqs[2]))

## One imaginary frequency confirms that the molecule is located at a saddle point.
## Visualized vector of that imaginary frequency shows a direction to one of the two energy minima.

In [ ]:
vis.vib(glob.glob('CNH.*.molden_normal_modes')[-1])

## Intrinsic reaction coordinate (IRC) method can be used to connect the transition state structure and either product or reactant 
## Optking can perfrom the IRC as well([Psi4/Optking manual](https://psicode.org/psi4manual/master/autodir_options_c/module__optking.html)).

In [ ]:
directions = ["forward", "backward"]
mol_info = TS.to_dict('psi4') # Save molecule information of the TS structure.

for direction in directions:
    print("IRC %s direction started" %direction)
    M = psi4.core.Molecule.from_dict(mol_info) # Define a molecule object based on the info saved previously.
    psi4.set_options({
        "opt_type":"IRC",
        "irc_direction":direction,
        "irc_step_size":0.1,
        "geom_maxiter": 100})
    psi4.optimize("SCF", molecule = M)



## Psi4 generated `irc_forward.xyz` and `irc_backward.xyz` files. You can visualize those using VMD.